# iMessage

本 Notebook 展示了如何使用 iMessage 聊天加载器。此类有助于将 iMessage 对话转换为 LangChain 聊天消息。

在 MacOS 上，iMessage 将对话存储在 `~/Library/Messages/chat.db` 的 sqlite 数据库中（至少对于 macOS Ventura 13.4）。
`IMessageChatLoader` 从该数据库文件加载。

1. 使用指向您想处理的 `chat.db` 数据库的文件路径创建 `IMessageChatLoader`。
2. 调用 `loader.load()`（或 `loader.lazy_load()`）来执行转换。您可以选择使用 `merge_chat_runs` 来合并同一发件人按顺序发送的消息，以及/或使用 `map_ai_messages` 将指定发件人的消息转换为“AIMessage”类。

## 1. 访问 Chat DB

您的终端很可能被拒绝访问 `~/Library/Messages`。要使用此类，您可以将 DB 复制到可访问的目录（例如 Documents）并从那里加载。或者（不推荐），您可以在“系统设置”>“安全性与隐私”>“完全磁盘访问权限”中为您的终端仿真器授予完全磁盘访问权限。

我们创建了一个您可以使用的示例数据库，位于[此链接的驱动器文件](https://drive.google.com/file/d/1NebNKqTA2NXApCmeH6mu0unJD2tANZzo/view?usp=sharing)。

In [1]:
# This uses some example data
import requests


def download_drive_file(url: str, output_path: str = "chat.db") -> None:
    file_id = url.split("/")[-2]
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"

    response = requests.get(download_url)
    if response.status_code != 200:
        print("Failed to download the file.")
        return

    with open(output_path, "wb") as file:
        file.write(response.content)
        print(f"File {output_path} downloaded.")


url = (
    "https://drive.google.com/file/d/1NebNKqTA2NXApCmeH6mu0unJD2tANZzo/view?usp=sharing"
)

# Download file to chat.db
download_drive_file(url)

File chat.db downloaded.


## 2. 创建聊天加载器

为加载器提供 zip 目录的文件路径。您可以选择指定映射到 ai 消息的用户 ID，以及配置是否合并消息运行。

In [2]:
from langchain_community.chat_loaders.imessage import IMessageChatLoader

In [3]:
loader = IMessageChatLoader(
    path="./chat.db",
)

## 3. 加载消息

`load()` (或 `lazy_load`) 方法返回一个 "ChatSessions" 列表，目前该列表仅包含每个已加载对话的消息列表。所有消息最初都被映射为 "HumanMessage" 对象。

您可以选择合并消息的 "runs"（来自同一发送者的连续消息）并选择一个发送者来代表 "AI"。经过微调的 LLM 将学习生成这些 AI 消息。

In [9]:
from typing import List

from langchain_community.chat_loaders.utils import (
    map_ai_messages,
    merge_chat_runs,
)
from langchain_core.chat_sessions import ChatSession

raw_messages = loader.lazy_load()
# Merge consecutive messages from the same sender into a single message
merged_messages = merge_chat_runs(raw_messages)
# Convert messages from "Tortoise" to AI messages. Do you have a guess who these conversations are between?
chat_sessions: List[ChatSession] = list(
    map_ai_messages(merged_messages, sender="Tortoise")
)

In [13]:
# Now all of the Tortoise's messages will take the AI message class
# which maps to the 'assistant' role in OpenAI's training format
chat_sessions[0]["messages"][:3]

[AIMessage(content="Slow and steady, that's my motto.", additional_kwargs={'message_time': 1693182723, 'sender': 'Tortoise'}, example=False),
 HumanMessage(content='Speed is key!', additional_kwargs={'message_time': 1693182753, 'sender': 'Hare'}, example=False),
 AIMessage(content='A balanced approach is more reliable.', additional_kwargs={'message_time': 1693182783, 'sender': 'Tortoise'}, example=False)]

## 3. 为微调做准备

现在是时候将我们的聊天消息转换为 OpenAI 字典了。我们可以使用 `convert_messages_for_finetuning` 工具来完成此操作。

In [14]:
from langchain_community.adapters.openai import convert_messages_for_finetuning

In [15]:
training_data = convert_messages_for_finetuning(chat_sessions)
print(f"Prepared {len(training_data)} dialogues for training")

Prepared 10 dialogues for training


## 4. 微调模型

现在是时候微调模型了。请确保已安装 `openai` 包，并且已正确设置 `OPENAI_API_KEY`。

In [16]:
%pip install --upgrade --quiet  langchain-openai

In [18]:
import json
import time
from io import BytesIO

import openai

# We will write the jsonl file in memory
my_file = BytesIO()
for m in training_data:
    my_file.write((json.dumps({"messages": m}) + "\n").encode("utf-8"))

my_file.seek(0)
training_file = openai.files.create(file=my_file, purpose="fine-tune")

# OpenAI audits each training file for compliance reasons.
# This make take a few minutes
status = openai.files.retrieve(training_file.id).status
start_time = time.time()
while status != "processed":
    print(f"Status=[{status}]... {time.time() - start_time:.2f}s", end="\r", flush=True)
    time.sleep(5)
    status = openai.files.retrieve(training_file.id).status
print(f"File {training_file.id} ready after {time.time() - start_time:.2f} seconds.")

File file-zHIgf4r8LltZG3RFpkGd4Sjf ready after 10.19 seconds.


文件准备就绪，是时候启动一个训练作业了。

In [19]:
job = openai.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo",
)

模型准备期间，请喝杯茶。这可能需要一些时间！

In [20]:
status = openai.fine_tuning.jobs.retrieve(job.id).status
start_time = time.time()
while status != "succeeded":
    print(f"Status=[{status}]... {time.time() - start_time:.2f}s", end="\r", flush=True)
    time.sleep(5)
    job = openai.fine_tuning.jobs.retrieve(job.id)
    status = job.status

In [21]:
print(job.fine_tuned_model)

ft:gpt-3.5-turbo-0613:personal::7sKoRdlz


## 5. 在 LangChain 中使用

您可以直接在 `ChatOpenAI` 模型类中使用生成的模型 ID。

In [22]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model=job.fine_tuned_model,
    temperature=1,
)

In [39]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are speaking to hare."),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [41]:
for tok in chain.stream({"input": "What's the golden thread?"}):
    print(tok, end="", flush=True)

A symbol of interconnectedness.